## Libraries

In [220]:
!pip3 install nltk
!pip3 install natsort
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yosef\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [343]:
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from natsort import natsorted
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer

## Read files

In [416]:
def read_files(file):
    if 'txt' in file:
        with open(f'docs/'+file, 'r', encoding='utf-8') as f:
            return f.read()

In [345]:
documents = []
for file in os.listdir('docs/'):
    documents.append(read_files(file))

# First Phase $:-$ 


In [346]:
token_docs = []
for document in documents:
    token_docs.append(word_tokenize(document))
token_docs
# documents


[['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser'],
 ['fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['antony', 'brutus', 'caeser', 'calpurnia'],
 ['mercy', 'worser'],
 ['brutus', 'caeser', 'mercy', 'worser'],
 ['caeser', 'mercy', 'worser'],
 ['antony', 'caeser', 'mercy'],
 ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where']]

## stemming


In [347]:
stemmer = PorterStemmer()
stemmed_docs = []
for tokens in token_docs:
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_docs.append(stemmed_tokens)

In [348]:
stemmed_docs

[['antoni', 'brutu', 'caeser', 'cleopatra', 'merci', 'worser'],
 ['fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['antoni', 'brutu', 'caeser', 'calpurnia'],
 ['merci', 'worser'],
 ['brutu', 'caeser', 'merci', 'worser'],
 ['caeser', 'merci', 'worser'],
 ['antoni', 'caeser', 'merci'],
 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angel', 'fool', 'in', 'rush', 'to', 'tread', 'where']]

In [407]:
documents

['antony brutus caeser cleopatra mercy worser',
 'fools fear in rush to tread where',
 'antony brutus caeser calpurnia ',
 'mercy worser',
 'brutus caeser mercy worser',
 'caeser mercy worser',
 'antony caeser mercy ',
 'angels fools fear in rush to tread where',
 'angels fools fear in rush to tread where',
 'angels fools in rush to tread where']

# Second phase $:-$

# Build positional index


In [406]:
pos_index = {}
document_number =0
natsorted(stemmed_docs)
for pos_doc, docu in enumerate(stemmed_docs):
    for pos_term, term in enumerate(docu):
        if term in pos_index:
            pos_index[term][0] = pos_index[term][0] + 1
            if document_number in pos_index[term][1]:
                pos_index[term][1][document_number].append(pos_term)
            else:
                pos_index[term][1][document_number] = [pos_term]
        else:
            pos_index[term] = []
            pos_index[term].append(1)
            pos_index[term].append({})     
            pos_index[term][1][document_number] = [pos_term]
    document_number += 1
pos_index

{'antoni': [3, {0: [0], 2: [0], 6: [0]}],
 'brutu': [3, {0: [1], 2: [1], 4: [0]}],
 'caeser': [5, {0: [2], 2: [2], 4: [1], 5: [0], 6: [1]}],
 'cleopatra': [1, {0: [3]}],
 'merci': [5, {0: [4], 3: [0], 4: [2], 5: [1], 6: [2]}],
 'worser': [4, {0: [5], 3: [1], 4: [3], 5: [2]}],
 'fool': [4, {1: [0], 7: [1], 8: [1], 9: [1]}],
 'fear': [3, {1: [1], 7: [2], 8: [2]}],
 'in': [4, {1: [2], 7: [3], 8: [3], 9: [2]}],
 'rush': [4, {1: [3], 7: [4], 8: [4], 9: [3]}],
 'to': [4, {1: [4], 7: [5], 8: [5], 9: [4]}],
 'tread': [4, {1: [5], 7: [6], 8: [6], 9: [5]}],
 'where': [4, {1: [6], 7: [7], 8: [7], 9: [6]}],
 'calpurnia': [1, {2: [3]}],
 'angel': [3, {7: [0], 8: [0], 9: [0]}]}

In [385]:
all_terms = []
for doc in pos_index:
    for term in doc.split():
        all_terms.append(term)
all_terms = set(all_terms)
all_terms

{'angel',
 'antoni',
 'brutu',
 'caeser',
 'calpurnia',
 'cleopatra',
 'fear',
 'fool',
 'in',
 'merci',
 'rush',
 'to',
 'tread',
 'where',
 'worser'}

## Term Frequency


In [408]:
def get_tf(document):
    wordDict = dict.fromkeys(all_terms, 0)
    for word in document:
        wordDict[word] += 1
    return wordDict



In [358]:
tf = pd.DataFrame(get_tf(stemmed_docs[0]).values(), index=get_tf(stemmed_docs[0]).keys())
for i in range(1, len(stemmed_docs)):
    tf[i] = get_tf(stemmed_docs[i]).values()
tf.columns = ['doc'+str(i) for i in range(1, 11)]

tf


,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
cleopatra,1,0,0,0,0,0,0,0,0,0
caeser,1,0,1,0,1,1,1,0,0,0
fear,0,1,0,0,0,0,0,1,1,0
antoni,1,0,1,0,0,0,1,0,0,0
fool,0,1,0,0,0,0,0,1,1,1
calpurnia,0,0,1,0,0,0,0,0,0,0
to,0,1,0,0,0,0,0,1,1,1
merci,1,0,0,1,1,1,1,0,0,0
brutu,1,0,1,0,1,0,0,0,0,0
where,0,1,0,0,0,0,0,1,1,1


## Weighted tf(1+ log tf)

In [359]:
def weighted_tf(x):
    if x > 0:
        return math.log10(x) + 1
    return 0

In [360]:
w_tf = tf.copy()
for i in range(0, len(stemmed_docs)):
    w_tf['doc'+str(i+1)] = tf['doc'+str(i+1)].apply(weighted_tf)

In [362]:
w_tf

,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
cleopatra,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
caeser,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
fear,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
antoni,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
fool,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
calpurnia,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
to,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
merci,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
brutu,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
where,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


## Inverse Document Frequency


In [367]:
idf = pd.DataFrame(columns=['df', 'idf'])
for i in range(len(tf)):
    in_term = w_tf.iloc[i].values.sum()
    idf.loc[i, 'df'] = in_term
    idf.loc[i, 'idf'] = math.log10(10 / (float(in_term)))
idf.index=w_tf.index

In [368]:
idf

,df,idf
cleopatra,1.0,1.0
caeser,5.0,0.30103
fear,3.0,0.522879
antoni,3.0,0.522879
fool,4.0,0.39794
calpurnia,1.0,1.0
to,4.0,0.39794
merci,5.0,0.30103
brutu,3.0,0.522879
where,4.0,0.39794


## TF.IDF

In [369]:
tf_idf = w_tf.multiply(tdf['idf'], axis=0)

In [370]:
tf_idf

,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
cleopatra,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
caeser,0.30103,0.0,0.30103,0.0,0.30103,0.30103,0.30103,0.0,0.0,0.0
fear,0.0,0.522879,0.0,0.0,0.0,0.0,0.0,0.522879,0.522879,0.0
antoni,0.522879,0.0,0.522879,0.0,0.0,0.0,0.522879,0.0,0.0,0.0
fool,0.0,0.39794,0.0,0.0,0.0,0.0,0.0,0.39794,0.39794,0.39794
calpurnia,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
to,0.0,0.39794,0.0,0.0,0.0,0.0,0.0,0.39794,0.39794,0.39794
merci,0.30103,0.0,0.0,0.30103,0.30103,0.30103,0.30103,0.0,0.0,0.0
brutu,0.522879,0.0,0.522879,0.0,0.522879,0.0,0.0,0.0,0.0,0.0
where,0.0,0.39794,0.0,0.0,0.0,0.0,0.0,0.39794,0.39794,0.39794


## Document length

In [371]:
def get_doc_len(col):
    return np.sqrt(tf_idf[col].apply(lambda x: x**2).sum())

In [375]:
doc_len = pd.DataFrame()
for col in tf_idf.columns:
    doc_len.loc[0, col+'_length']= get_doc_len(col)
doc_len

,doc1_length,doc2_length,doc3_length,doc4_length,doc5_length,doc6_length,doc7_length,doc8_length,doc9_length,doc10_length
0,1.373462,1.106137,1.279618,0.498974,0.782941,0.582747,0.67427,1.223496,1.223496,1.106137


In [376]:
doc_len['doc1_length'].values[0]

1.3734623153231016

## Normalized TF.IDF

In [377]:
def get_norm_tf_idf(col, x):
    try:
        return x / doc_len[col+'_length'].values[0]
    except:
        return 0

In [ ]:
norm_tf_idf = pd.DataFrame()
for col in tf_idf.columns:
    norm_tf_idf[col] = tf_idf[col].apply(lambda x : get_norm_tf_idf(col, x))
norm_tf_idf

# Similarity 

In [412]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pos_index)
X = X.T.toarray()
df = pd.DataFrame(X, index=vectorizer.get_feature_names_out())

In [414]:
def get_similar_articles(q, df):
    print("query:", q)
    # Convert the query become a vector
    query = [q]
    q_vec = vectorizer.transform(query).toarray().reshape(df.shape[0],)
    sim = {}  # Calculate the similarity
    for i in range(10):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)

    # Sort the values 
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    # Print the articles and their similarity values
    for doc, score in sim_sorted:
        if score > 0.5:
            print("Similarity value:", score)
            print("The article is:", doc+1)

    # print("Similarity value:", sim_sorted[0][1])
    # print("sim =", sim_sorted[0])
    # print("doc =", documents)
    # print("The article is:", documents[sim_sorted[0][0]])
    print('\t\t\t\tf-raw')
    print(tf.loc[q.split()])

    print('\t\t\t\tw tf(1+ log tf)')
    print(w_tf.loc[q.split()])

    print('\t\t\t\tidf')
    print(tdf.loc[q.split()])

    print('\t\t\t\ttf*idf')
    print(tf_idf.loc[q.split()])

    print('\t\t\t\tnormalized')
    print(norm_tf_idf.loc[q.split()])

In [ ]:
q1 = 'antony brutus'
get_similar_articles(q1, df)

## Input Query

In [402]:
def get_w_tf(x):
    try:
        return math.log10(x)+1
    except:
        return 0

In [400]:
def insert_query(q):
    docs_found = put_query(q, 2)
    if docs_found == []:
        return "Not Fount"
    new_q = preprocessing(q)
    query = pd.DataFrame(index=norm_tf_idf.index)
    query['tf'] = [1 if x in new_q else 0 for x in list(norm_tf_idf.index)]
    query['w_tf'] = query['tf'].apply(lambda x : get_w_tf(x))
    product = norm_tf_idf.multiply(query['w_tf'], axis=0)
    query['idf'] = tdf['idf'] * query['w_tf']
    query['tf_idf'] = query['w_tf'] * query['idf']
    query['normalized'] = 0
    for i in range(len(query)):
        query['normalized'].iloc[i] = float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))
    print('Query Details')
    print(query.loc[new_q])
    product2 = product.multiply(query['normalized'], axis=0)
    scores = {}
    for col in put_query(q, 2):
            scores[col] = product2[col].sum()
    product_result = product2[list(scores.keys())].loc[new_q]
    print()
    print('Product (query*matched doc)')
    print(product_result)
    print()
    print('product sum')
    print(product_result.sum())
    print()
    print('Query Length')
    q_len = math.sqrt(sum([x**2 for x in query['idf'].loc[new_q]]))
    print(q_len)
    print()
    print('Cosine Simliarity')
    print(product_result.sum())
    print()
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    print('Returned docs')
    for typle in sorted_scores:
        print(typle[0], end=" ")

In [ ]:
insert_query('antony the brutus')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def process_query(query):
    query_tokens = [stemmer.stem(token) for token in word_tokenize(query)]
    query_vector = np.zeros((1, len(pos_index)))
    for i, term in enumerate(pos_index.keys()):
        query_vector[0, i] = query_tokens.count(term) * idf[term]
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix)
    ranked_documents = np.argsort(similarity_scores)[0][::-1]
    return ranked_documents

# Example usage
query = "information retrieval"
ranked_documents = process_query(query)
print("Ranked Documents:", ranked_documents)


In [398]:
def put_query(q, display=1):
    lis = [[] for i in range(10)]
    q = preprocessing(q)
    for term in q:
        if term in pos_index.keys():
            for key in pos_index[term][1].keys():
                if lis[key-1] != []:
                    if lis[key-1][-1] == pos_index[term][1][key][0]-1:
                        lis[key-1].append(pos_index[term][1][key][0])
                else:
                    lis[key-1].append(pos_index[term][1][key][0])
    positions = []
    if display==1:
        for pos, list in enumerate(lis, start=1):
            if len(list) == len(q):
                positions.append('document '+str(pos))
        return positions
    else:
        for pos, list in enumerate(lis, start=1):
            if len(list) == len(q):
                positions.append('doc'+str(pos))
        return positions